# R5.C.08 - Analyse de données : ACP, AFC

## 1. Codesource, fichier(s) de données ou un lien de récupération


Le code source et le jeu de données sont disponibles sur GitHub :
- [https://github.com/Arcthuruss/Glue-factory-analysis](https://github.com/Arcthuruss/Glue-factory-analysis)
**Instruction** : Git LFS est requis pour cloner le dépôt car le jeu de données est volumineux (=448Mo).

---

## 2. Introduction, constitution du groupe

Le projet consiste à analyser un jeu de données de notre choix en utilisant des techniques d'analyse de données vues lors de la ressource R5.C.08 - Techniques d'intelligence artificielle.
Avec pour objectif de pratiquer les techniques d'ACP (Analyse en Composantes Principales) et d'AFC (Analyse Factorielle des Correspondances) sur des variables quantitatives et qualitatives respectivement.

Le projet a été réalisé par un groupe de deux personnes :
- DONNARD Luc
- NÉVOT Pierre

---

## 3. Description du jeu de données

Le jeu de données provient de Kaggle : [https://www.kaggle.com/datasets/takamotoki/jra-horse-racing-dataset](https://www.kaggle.com/datasets/takamotoki/jra-horse-racing-dataset)

Il contient plusieurs csv tel que :
- 19860105-20210731_laptime.csv qui contient les données des tours de chaque course entre 1986 et 2021
- 19860105-20210731_odds.csv qui contient les cotes des chevaux pour chaque course entre 1986 et 2021
- 19860105-20210731_race_results.csv qui contient les résultats de chaque course entre 1986 et 2021
- 20020615-20210731_corner_passing_order.csv qui contient les positions des chevaux à chaque virage entre 2002 et 2021

On a choisi d'utiliser 19860105-20210731_race_results.csv car il contient des informations sur le resultat du cheval gagnant ainsi que pleins d'informations sur la course.
Le dataset contient au minimun 1 554 146 entrées et 66 colonnes au total.

En voici les principales variables :
- Turf and Dirt Category : Catégorie de la piste (herbe ou terre)
- Clockwise And Anti-clockwise and Straight Course Category : Catégorie de la course (sens horaire, antihoraire ou ligne droite)
- Distance(m) : Distance de la course en mètres
- Weather : Conditions météorologiques
- Track Condition1 : État de la piste
- Final Position : Position finale du cheval
- Bracket Number : Numéro de la série
- Post Position : Position de départ
- Horse Name : Nom du cheval
- Age : Âge du cheval
- Jockey : Nom du jockey
- Total Time(1/10s) : Temps total en dixièmes de seconde
- Position 3rd Corner : Position au 3ème virage
- Position 4th Corner : Position au 4ème virage
- Win Odds(100Yen) : Cote de victoire (en 100 Yen)
- Win Fav : Favori à la victoire

Le contexte est l'analyse des courses de chevaux au Japon, en utilisant des données historiques pour identifier des tendances et des facteurs influençant les résultats des courses.

---

## 4. Nettoyage de données

On a retiré les colonnes non pertinentes qui contenaient des valeurs non pertinentes et incomplètes.
On a ensuite traduit les attributs importants du japonais vers l'anglais pour faciliter l'analyse.
Tel que le type de course, si la course est dans le sens des aiguilles d'une montre ou non, la météo, l'état de la piste.
A part cela le dataset étant propre on a 1 554 146 entrées de données complètes.

In [13]:
# imports du projet
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

time_str_to_ms = lambda time_as_str : [(int(i[0])*600 + int(i[1])*10 + int(i[2])) for i in [i.replace(':', ' ').replace('.', ' ').split(' ') for i in time_as_str]]

In [ ]:
# filter script

input_file = "./datasets/19860105-20210731_race_result.csv"
output_file = "./clean_datasets/filtered_race_result.csv"

# Ligne qui va être supprimée (pas de nom de colonne car le header est en japonais et prendrais trop de temps)
positions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 40, 41, 43, 47, 49, 52, 53, 54, 57, 60, 61, 62, 63, 64, 65]

with open(input_file, newline='', encoding='utf-8') as infile, \
    open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for row in reader:
        filtered_row = [col for i, col in enumerate(row) if i not in positions]
        if all(filtered_row):
            writer.writerow(filtered_row)

# Remplace le header par le header en anglais
with open(output_file, 'r', newline='', encoding='utf-8') as infile:
    lines = infile.readlines()
header = "Turf and Dirt Category,Clockwise And Anti-clockwise and Straight Course Category,Distance(m),Weather,Track Condition1,Final Position,Bracket Number,Post Position,Horse Name,Age,Jockey,Total Time(1/10s),Position 3rd Corner,Position 4th Corner,Win Odds(100Yen),Win Fav"
lines[0] = header + "\n"
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    outfile.writelines(lines)


In [4]:
# Traduit les noms propres jopnais en anglais

# Fonction pour charger le cache
def load_cache(cache_file):
    try:
        with open(cache_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

# Fonction pour sauvegarder le cache
def save_cache(cache, cache_file):
    with open(cache_file, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=4)

# Si la valeur n'est pas dans le cache, demande la traduction
def ask_translation(text):
    print(f"Traduction pour '{text}': ", end="")
    return input().strip()

def annotate_csv(input_file, output_file, cache_file='translation_cache.json'):
    cache = load_cache(cache_file)
    df = pd.read_csv(input_file)

    # Ne traiter que les 5 premières colonnes
    cols_to_check = df.columns[:5]

    for col in cols_to_check:
        for i, value in enumerate(df[col]):
            if pd.isna(value):
                continue
            # Vérifier si la valeur est du texte japonais et non déjà dans le cache
            if value not in cache and any('\u3040' <= char <= '\u30ff' or '\u4e00' <= char <= '\u9faf' for char in str(value)):
                translation = ask_translation(value)
                cache[value] = translation

    save_cache(cache, cache_file)

def replace_with_cache(df, cache):
    for col in df.columns[:5]:
        df[col] = df[col].apply(lambda x: cache.get(x, x) if pd.notna(x) else x)
    return df

input_file = "./clean_datasets/filtered_race_result.csv"
output_file = "./clean_datasets/translated_race_result.csv"
annotate_csv(input_file, output_file)
cache = load_cache('translation_cache.json')
df = pd.read_csv(input_file)
df = replace_with_cache(df, cache)
df.to_csv(output_file, index=False)

In [11]:
translated_file = "./clean_datasets/translated_race_result.csv"
df = pd.read_csv(translated_file)

---

## 5. Variables quantitatives : ACP

### 5.1 Standardisation












In [41]:
distance = df.iloc[:, 2].values
final_position = df.iloc[:, 5].values
bracket_number = df.iloc[:, 6].values
post_position = df.iloc[:, 7].values
age = df.iloc[:, 9].values
third_corner_position = df.iloc[:, 12].values
fourth_corner_position = df.iloc[:, 13].values
win_odds = df.iloc[:, 14].values
win_fav = df.iloc[:, 15].values


### 5.2 Entraînement du modèle

Effectuez une ACP (Analyse en Composantes Principales) sur les données standardisées.

In [43]:
pca = PCA(n_components=2)
print(pca.fit_transform(final_position))

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.  2.  3. ... 14. 15. 16.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

### 5.3 Choix du nombre de composantes principales

Expliquez le critère de choix (ex : seuil de variance expliquée).

### 5.4 Tableau de valeurs singulières + % variance

Présentez un tableau avec les valeurs propres et le pourcentage de variance expliquée par chaque composante.

### 5.5 Visualisation des variables avec biplot, étude sur la corrélation entre variables

Affichez un biplot et commentez la corrélation entre variables.

### 5.6 Visualisation des individus avec le plan factoriel

Projetez les individus sur le plan des deux premières composantes principales.

---

## 6. Variables qualitatives : AFC

### 6.1 Choix de deux variables, tableau de contingence

Sélectionnez deux variables qualitatives et construisez leur tableau de contingence.

### 6.2 Test de sphéricité de Bartlett/Test de Chi2, conclusion sur la corrélation

Effectuez le test du Chi2 sur le tableau de contingence et concluez sur la corrélation.

### 6.3 Standardisation

Expliquez la standardisation pour l'AFC/AMC si nécessaire.

### 6.4 Entraînement du modèle FactorAnalyzer

Entraînez un modèle FactorAnalyzer sur les données qualitatives.

### 6.5 ScreePlot pour les valeurs propres de facteurs

Affichez le scree plot des valeurs propres.

### 6.6 Graphiques sur 3 rotations différentes

Visualisez les résultats de l'analyse factorielle avec trois types de rotations différentes (ex : varimax, quartimax, equamax).

---